# WordNet

**WordNet :** base de données lexicales pour l’anglais.

NLTK fournit une interface d’interrogation.

## Les Synsets de WordNet

Liste de *Synsets* pour un terme donné.

**Synsets** : ensembles de synonymes pour chaque acception.

Souvent, un seul *Synset*.

In [ ]:
# Import Wordnet
import nltk
from nltk.corpus import wordnet

In [ ]:
# Synsets of the word "Duck"
syn = wordnet.synsets('duck')
print(syn)

Pour chaque *synset* d’un terme, on peut obtenir :
- sa définition
- des exemples d’utilisation
- son étiquette grammaticale (*n*, *a*, *r* ou *v*)
- la liste de ses hyponymes et de ses hyperonymes
- les plus proches hyperonymes communs avec un autre *synset*
- ses lemmes

In [ ]:
definition = syn[0].definition()
examples = syn[4].examples()
tag = syn[0].pos()
hypernyms = syn[0].hypernyms()
hyponyms = syn[0].hyponyms()
lemmas = syn[7].lemmas()
related = syn[0].lowest_common_hypernyms(wordnet.synset('boat.n.01'))

### Rechercher les synonymes

Les lemmes d’un *Synset* se concevant comme des synonymes, l’une des finalités envisageables serait par exemple d’en obtenir la liste pour une certaine catégorie grammaticale :

In [ ]:
def get_synonyms(word, pos=None):
    """Lists all possible synonyms of a word, except the word itself.
    
    Keyword arguments:
    word -- the word to look up
    pos -- POS-tagging
    """
    synonyms = set()
    synsets = wordnet.synsets(word, pos=pos)
    for synset in synsets:
        [synonyms.add(lemma.name()) for lemma in synset.lemmas()]
    synonyms.remove(word)
    return synonyms

synonyms = get_synonyms('duck', 'v')
print(synonyms)

### Rechercher les antonymes

À l’inverse, on peut aussi bien lister les antonymes d’un *Synset* particulier :

In [ ]:
def get_antonyms(synset):
    """Lists the antonyms of all the possible synonyms of a
    particular Synset.
    
    Keyword argument:
    synset -- the given synset
    """
    antonyms = set()
    for lemma in synset.lemmas():
        [antonyms.add(antonym.name()) for antonym in lemma.antonyms()]
    return antonyms

kind = wordnet.synset('kind.a.01')
antonyms = get_antonyms(kind)

### Comparer des *synsets*

L’accès aux hyperonymes et aux hyponymes d’un *synset* permet de :
- remonter jusqu’à l’hyperonyme racine
- obtenir la liste des *synsets* voisins
- calculer la similarité lexicale entre deux *synsets*

In [ ]:
# Start synset
duck = wordnet.synset('duck.n.01')

Comment obtenir la liste des *synsets* voisins ?

In [ ]:
duck_hypernym = duck.hypernyms()[0]
duck_related = duck_hypernym.hyponyms()
print(duck_related)

Comment remonter jusqu’à l’hyperonyme racine ?

In [ ]:
path_to_duck = duck.hypernym_paths()
print(path_to_duck)

Comment calculer la similitude entre deux *synsets* ?

In [ ]:
duck = wordnet.synset('duck.n.01')
goose = wordnet.synset('goose.n.01')
whale = wordnet.synset('whale.n.01')
boat = wordnet.synset('boat.n.01')

Les différents calculs de similarité lexicale se basent sur la plus courte distance entre les *synsets* et un hyperonyme commun :

In [ ]:
goose.shortest_path_distance(duck)

Quelques calculs de similarité lexicale :

In [ ]:
# Based on the path
path_sim = duck.path_similarity(goose)
# Leacock-Chordorow similarity
lch_sim = duck.lch_similarity(goose)
# Wu-Palmer similarity
wu_sim = duck.wup_similarity(goose)

Les scores peuvent fortement varier selon la méthode utilisée :

- **Leacock-Chordorow :** renvoie un score de similarité lexicale entre deux mots, basé d’une part sur le plus court chemin qui connecte leurs sens et d’autre part sur la profondeur maximale de ces sens dans la taxonomie.

- **Wu-Palmer :** renvoie un score de similarité lexicale entre les sens de deux mots, basé sur leur profondeur dans la taxonomie ainsi que sur celle de leur ancêtre le plus spécifique (*Least Common Subsumer*).

Comme ces calculs reposent sur des hyperonymes communs, ils ne sont pas opportuns :
- pour des termes employés dans des contextes grammaticaux différents (*duck*/nom, *duck*/verbe)
- pour la similarité entre deux verbes (peu ont un hyperonyme commun)

La réponse renvoyée sera de type `None`

## Un WordNet pour le français ?

**WOLF :** Wordnet Libre du Français
- depuis 2008
- licence libre CeCILL
- développé à l’Inria
- basé sur la traduction du WordNet de Princeton

WOLF est intégré à WordNet :

In [ ]:
import nltk
from nltk.corpus import wordnet
synsets = wordnet.synsets('canard', lang='fra')
print(synsets)

Comme d’autres langues dans le cadre du projet *Open Multilingual WordNet* :

In [ ]:
print(wordnet.langs())

Le paramètre `lang` s’emploie avec les méthodes :
- `synsets()` : obtenir la liste des *synsets* d’une langue
- `lemma_names()` : obtenir la liste des lemmes pour un *synset*
- `lemma()` : construire une instance `Lemma`
- `words()` : obtenir une liste des mots du WordNet

In [ ]:
duck_lemmas = wordnet.synset('duck.n.01').lemma_names('fra')
lemma = wordnet.lemma('duck.n.01.canard', lang='fra').name()
words_fr = wordnet.words('fra')

Quelques ajustements permettent de soutenir une recherche de synonymes multilingue :

In [ ]:
def get_synonyms(word, pos=None, lang='eng'):
    """Lists all possible synonyms of a word, except the word itself
    for a given language.
    
    Keyword arguments:
    word -- the word to look up
    pos -- POS-tagging
    lang -- language parameter to translate
    """
    synonyms = set()
    synsets = wordnet.synsets(word, pos=pos, lang=lang)
    for synset in synsets:
        [synonyms.add(lemma.name()) for lemma in synset.lemmas(lang)]
    try:
        synonyms.remove(word)
        return synonyms
    except KeyError:
        postags = {"n": "noun", "a": "adjective", "v":"verb", "r": "adverb"}
        return f'There is no match for "{word}" as {postags[pos]} in "{lang}" WordNet version.'

synonyms = get_synonyms('barca', 'n', 'ita')